In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import gmaps
from pprint import pprint
from config import key_dict

In [2]:
weather_df = pd.read_csv('Resources/weather_dataframe.csv')
weather_df.head()

,Lat,Long,City,Country,Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph)
0,71.9667,102.5000,Khatanga,RU,-25.33,95,100,7.74
1,-35.0169,117.8917,Albany,AU,91.40,22,99,4.61
2,15.4510,36.4000,Kassala,SD,67.08,85,0,11.79
3,38.5167,-28.7000,Ribeira Grande,PT,60.30,88,100,20.22
4,65.6145,-37.6368,Tasiilaq,GL,24.80,54,75,3.44


In [3]:
gmaps.configure(api_key=key_dict['google'])

In [4]:
fig = gmaps.figure()
locations = weather_df[['Lat','Long']]
humidity_percent = weather_df['Humidity(%)'].astype(float)


humid_layer = gmaps.heatmap_layer(locations, weights = humidity_percent,
                                 dissipating = False, max_intensity = humidity_percent.max()*1.5,
                                 point_radius = 4)

fig.add_layer(humid_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
ideal_locations = weather_df.loc[(weather_df['Temp(F)'] >= 55) & (weather_df['Temp(F)'] <= 75),:]
ideal_locations = ideal_locations.loc[ideal_locations['Humidity(%)'] < 50,:]
ideal_locations.reset_index(inplace = True, drop = True)
ideal_locations

,Lat,Long,City,Country,Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph)
0,20.5169,-13.0499,Atar,MR,67.28,23,0,13.87
1,23.9912,-104.6749,Constitucion,MX,64.40,12,1,6.91
2,22.8909,-109.9124,Cabo San Lucas,MX,66.51,45,1,8.43
3,-42.7826,147.0587,New Norfolk,AU,72.34,41,59,3.00
4,25.0213,37.2685,Umm Lajj,SA,72.93,42,0,12.77
5,-42.8794,147.3294,Hobart,AU,72.61,40,20,11.50
6,33.5959,56.9244,Tabas,IR,67.41,17,47,9.15
7,-43.3002,-65.1023,Rawson,AR,66.79,30,31,16.73
8,-45.8667,-67.5000,Comodoro Rivadavia,AR,60.80,44,0,21.85
9,25.3500,-102.8000,Viesca,MX,73.40,18,1,6.17


In [75]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"
key = key_dict['google']

hotel_locations = []
hotel_dict_list = []


for ind,dat in ideal_locations.iterrows():
    params = {
        'key':key,
        'location':f"{dat['Lat']},{dat['Long']}",
        'radius':5000,
        'query':'Hotel'
    }
    
    information = requests.get(base_url,params).json()
    try:
        coords = information['results'][0]['geometry']['location']
        hotel_locations.append([coords['lat'],coords['lng']])
        hotel_dict_list.append({'name':information['results'][0]['name'],
                                'city':ideal_locations.iloc[ind,2],
                                'country':ideal_locations.iloc[ind,3]})
    except IndexError:
        continue

    
info_box_template = """

<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Country</dt><dd>{country}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>

"""
hotel_info = [info_box_template.format(**hotel) for hotel in hotel_dict_list]

pin_layer = gmaps.marker_layer(hotel_locations, 
                               info_box_content = hotel_info)
fig.add_layer(pin_layer)
fig

Figure(layout=FigureLayout(height='420px'))